In [ ]:
import pandas as pd

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)

pd.set_option('display.width', 3000)

# DATA_DIR = r'c:\Users\ping\Files_win10\python\py311\stocks\data'
# Manually construct the full path before loading
full_file_path = r'c:\Users\ping\Files_win10\python\py311\stocks\data\df_OHLCV_clean_stocks_etfs.parquet'
df_OHLCV = pd.read_parquet(full_file_path, engine='pyarrow')

In [4]:
df_OHLCV.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 374000 entries, ('A', Timestamp('2024-09-23 00:00:00')) to ('ZWS', Timestamp('2025-09-22 00:00:00'))
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Adj Open   374000 non-null  float64
 1   Adj High   374000 non-null  float64
 2   Adj Low    374000 non-null  float64
 3   Adj Close  374000 non-null  float64
 4   Volume     374000 non-null  int64  
dtypes: float64(4), int64(1)
memory usage: 15.7+ MB


############################################

In [5]:
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import pprint # Import the pretty-print library

def plot_interactive_performers_widget(df_ohlcv, 
                                       default_metric='Price',
                                       default_period='1Y', 
                                       default_rank_start=1,
                                       default_rank_end=10):
    """
    Creates a robust interactive plot using ipywidgets for state management.
    Allows for selecting tickers by a slice of their rank (e.g., 10th to 20th).
    Returns a list that is updated with the currently displayed tickers.
    """
    # --- [Section 1 & 2: Data Prep & Pre-computation] ---
    print("Reshaping and pre-computing all performance data...")
    if not isinstance(df_ohlcv.index, pd.MultiIndex) or 'Adj Close' not in df_ohlcv.columns:
        raise ValueError("Expected 'long' format with (Ticker, Date) MultiIndex and 'Adj Close' column.")
    df_close = df_ohlcv['Adj Close'].unstack(level=0)
    df_close.index = pd.to_datetime(df_close.index)
    end_date = df_close.index.max()
    metrics = ['Price', 'Sharpe']
    periods = {
        '1D': end_date - pd.DateOffset(days=1), '5D': end_date - pd.DateOffset(days=5),
        '3M': end_date - pd.DateOffset(months=3), '6M': end_date - pd.DateOffset(months=6),
        'YTD': datetime(end_date.year, 1, 1), '1Y': end_date - pd.DateOffset(years=1),
        '5Y': end_date - pd.DateOffset(years=5),
    }
    all_rankings = {metric: {} for metric in metrics}
    period_normalized_data = {}
    for label, start_date in periods.items():
        period_df = df_close.loc[start_date:end_date]
        if period_df.empty or len(period_df) < 2: continue
        first_prices, last_prices = period_df.bfill().iloc[0], period_df.ffill().iloc[-1]
        all_rankings['Price'][label] = (last_prices / first_prices).dropna().sort_values(ascending=False).index.tolist()
        daily_returns = period_df.pct_change()
        mean_returns, std_returns = daily_returns.mean(), daily_returns.std()
        sharpe_ratio = (mean_returns / std_returns).fillna(0)
        all_rankings['Sharpe'][label] = (sharpe_ratio * np.sqrt(252)).sort_values(ascending=False).index.tolist()
        period_normalized_data[label] = period_df.div(first_prices)
    print("Pre-computation finished.")

    # --- 3. Create ipywidgets Controls & Data Holder ---
    rank_options = [1, 5, 10, 20, 30, 40, 50, 75, 100]
    max_traces = 50 
    metric_dropdown = widgets.Dropdown(options=metrics, value=default_metric, description='Metric:')
    period_dropdown = widgets.Dropdown(options=list(periods.keys()), value=default_period, description='Period:')
    rank_start_dropdown = widgets.Dropdown(options=rank_options, value=default_rank_start, description='Rank Start:')
    rank_end_dropdown = widgets.Dropdown(options=rank_options, value=default_rank_end, description='Rank End:')
    update_button = widgets.Button(description="Update Chart", button_style='primary')
    ticker_list_output = widgets.Output()
    
    _currently_displayed_tickers = []

    # --- 4. Create a FigureWidget ---
    fig = go.FigureWidget()
    for i in range(max_traces):
        fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name=f'placeholder_{i}', visible=False, showlegend=False))

    # --- 5. Define the Update Logic ---
    def update_plot(button_click):
        metric, period, rank_start, rank_end = metric_dropdown.value, period_dropdown.value, rank_start_dropdown.value, rank_end_dropdown.value
        
        ticker_list_output.clear_output()
        
        if rank_start > rank_end:
            with ticker_list_output: print("Error: 'Rank Start' must be less than or equal to 'Rank End'.")
            return
        
        ranked_tickers = all_rankings[metric].get(period, [])
        tickers_to_display = ranked_tickers[rank_start-1:rank_end]
        norm_data = period_normalized_data.get(period)

        with fig.batch_update():
            for i in range(max_traces):
                trace = fig.data[i]
                if i < len(tickers_to_display) and norm_data is not None:
                    ticker = tickers_to_display[i]
                    trace.x, trace.y, trace.name = norm_data[ticker].index, norm_data[ticker], ticker
                    trace.visible, trace.showlegend = True, True
                else:
                    trace.visible, trace.showlegend = False, False
        
        _currently_displayed_tickers.clear()
        _currently_displayed_tickers.extend(tickers_to_display)
        
        with ticker_list_output:
            print("Currently Displayed Tickers:")
            # --- NEW: Use compact=True to force a more horizontal layout ---
            pprint.pprint(tickers_to_display, width=120, compact=True) 

    # --- 6. Set up Initial View and Layout ---
    update_plot(None)
    fig.update_layout(
        title_text='Top Performing Tickers (Normalized to 1)',
        xaxis_title='Date', yaxis_title='Normalized Price (Start = 1)',
        hovermode='x unified', legend_title_text='Tickers',
        height=700, margin=dict(t=50)
    )
    fig.add_hline(y=1, line_width=1, line_dash="dash", line_color="grey")
    update_button.on_click(update_plot)
    
    # --- 7. Display the UI ---
    controls = widgets.HBox([metric_dropdown, period_dropdown, rank_start_dropdown, rank_end_dropdown, update_button])
    ui_container = widgets.VBox([controls, ticker_list_output], layout=widgets.Layout(margin='10px 0 20px 0'))
    
    display(ui_container, fig)
    
    return _currently_displayed_tickers

In [21]:
# --- NEW: Initialize the variable before the call ---
plotted_tickers = [] 

# Call the final function and capture the returned list
# The 'plotted_tickers' list will be updated in place every time you click the button.
plotted_tickers = plot_interactive_performers_widget(
    df_OHLCV, 
    default_metric='Sharpe', 
    default_period='1Y', 
    default_rank_start=20, 
    default_rank_end=40
)

# --- NEW: Add a comment explaining how to use the variable ---
# After interacting with the plot and clicking "Update Chart", you can run a separate cell
# to see the contents of 'plotted_tickers'.
# For example, in a new cell, just type:
# print(plotted_tickers)

print("\n--- After the plot, the 'plotted_tickers' variable holds the last displayed list: ---")
print(plotted_tickers)

Reshaping and pre-computing all performance data...
Pre-computation finished.


FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'ESLT',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'cf50796d-bbcd-47d7-86e4-164375636bdf',
              'visible': True,
              'x': array([datetime.datetime(2024, 9, 23, 0, 0),
                          datetime.datetime(2024, 9, 24, 0, 0),
                          datetime.datetime(2024, 9, 25, 0, 0), ...,
                          datetime.datetime(2025, 9, 18, 0, 0),
                          datetime.datetime(2025, 9, 19, 0, 0),
                          datetime.datetime(2025, 9, 22, 0, 0)], dtype=object),
              'y': array([1.        , 1.01230212, 1.01178231, ..., 2.57609409, 2.56212754,
                          2.61988396])},
             {'mode': 'lines',
              'name': 'SPSB',
              'showlegend': True,
              'type': 'scatter',
              'uid': '770c0fb5-b3e7-4d30-9f2a-fe4d4856a72d',
              'visible': True,
        


--- After the plot, the 'plotted_tickers' variable holds the last displayed list: ---
['ESLT', 'SPSB', 'SGHC', 'VRNA', 'SHLD', 'STIP', 'OKLO', 'VTIP', 'KTOS', 'IDCC', 'HOOD', 'LLYVK', 'CLS', 'JPIE', 'LLYVA', 'PLTR', 'RBLX', 'RKLB', 'SOFI', 'FLOT', 'KGC']


In [10]:
print("\n--- After the plot, the 'plotted_tickers' variable holds the last displayed list: ---")
print(plotted_tickers)


--- After the plot, the 'plotted_tickers' variable holds the last displayed list: ---
['HOOD', 'LLYVK', 'CLS', 'JPIE', 'LLYVA', 'PLTR', 'RBLX', 'RKLB', 'SOFI', 'FLOT', 'KGC', 'SRAD', 'SAN', 'LTM', 'DB', 'TPR', 'SPTS', 'GFI', 'VGSH', 'IONQ', 'DFSD']


############################################

In [15]:
import pandas as pd
import numpy as np
from IPython.display import display, Markdown

def inspect_ticker_data(df_ohlcv, ticker, period):
    """
    Performs a detailed calculation sanity check for a single ticker over a given period.

    Args:
        df_ohlcv (pd.DataFrame): The original 'long' format DataFrame with a 
                                 (Ticker, Date) MultiIndex.
        ticker (str): The ticker symbol to inspect (e.g., 'NVDA').
        period (str): The period to analyze (e.g., '1Y', '3M', '5D').
    """
    # --- 1. Data Preparation ---
    
    # Check if the ticker exists
    if ticker not in df_ohlcv.index.get_level_values(0):
        print(f"Error: Ticker '{ticker}' not found in the DataFrame.")
        return

    # Select all data for the specified ticker and create a clean DataFrame
    ticker_df = df_ohlcv.loc[ticker].copy()
    ticker_df.index = pd.to_datetime(ticker_df.index)
    
    # Define the start date based on the period
    end_date = ticker_df.index.max()
    periods = {
        '1D': end_date - pd.DateOffset(days=1), '5D': end_date - pd.DateOffset(days=5),
        '3M': end_date - pd.DateOffset(months=3), '6M': end_date - pd.DateOffset(months=6),
        'YTD': datetime(end_date.year, 1, 1), '1Y': end_date - pd.DateOffset(years=1),
        '5Y': end_date - pd.DateOffset(years=5),
    }
    
    if period not in periods:
        print(f"Error: Period '{period}' is not a valid option. Choose from {list(periods.keys())}")
        return
        
    start_date = periods[period]
    
    # Slice the DataFrame for the requested period
    period_df = ticker_df.loc[start_date:end_date].copy()

    if len(period_df) < 2:
        print(f"Not enough data for ticker '{ticker}' in period '{period}' to perform calculations.")
        return

    # --- 2. Perform and Display Calculations ---

    display(Markdown(f"### Sanity Check for Ticker: `{ticker}` | Period: `{period}`"))

    # Price Performance
    start_price = period_df['Adj Close'].bfill().iloc[0]
    end_price = period_df['Adj Close'].ffill().iloc[-1]
    normalized_price = end_price / start_price
    
    display(Markdown("**1. Price Performance Calculation:**"))
    print(f"   - Start Date ({period_df.index.min().date()}): {start_price:,.2f}")
    print(f"   - End Date   ({period_df.index.max().date()}): {end_price:,.2f}")
    print(f"   - Final Normalized Price: ({end_price:,.2f} / {start_price:,.2f}) = {normalized_price:.4f}\n")

    # Sharpe Ratio
    period_df['Daily Return'] = period_df['Adj Close'].pct_change()
    mean_daily_return = period_df['Daily Return'].mean()
    std_daily_return = period_df['Daily Return'].std()
    
    # Handle case where standard deviation is zero (flat price)
    daily_sharpe = 0 if std_daily_return == 0 else mean_daily_return / std_daily_return
    annualized_sharpe = daily_sharpe * np.sqrt(252)

    display(Markdown("**2. Sharpe Ratio Calculation:**"))
    print(f"   - Number of Trading Days: {len(period_df)}")
    print(f"   - Mean of Daily Returns: {mean_daily_return:.6f}")
    print(f"   - Std Dev of Daily Returns: {std_daily_return:.6f}")
    print(f"   - Daily Sharpe Ratio: ({mean_daily_return:.6f} / {std_daily_return:.6f}) = {daily_sharpe:.4f}")
    print(f"   - Annualized Sharpe Ratio: ({daily_sharpe:.4f} * sqrt(252)) = {annualized_sharpe:.4f}\n")
    
    # --- 3. Display Underlying Data ---
    display(Markdown("**3. Underlying Data Sample:**"))
    display(period_df[['Adj Close', 'Daily Return']].head())
    if len(period_df) > 10:
        print("   ...")
        display(period_df[['Adj Close', 'Daily Return']].tail())
        
    return period_df[['Adj Close', 'Daily Return']]


# # === Example Usage ===
# # Assuming 'long_format_df' is the sample DataFrame we created before.
# if __name__ == '__main__':
#     # (Using the sample data generation from the previous step)
#     date_rng = pd.date_range(start='2020-01-01', end='2023-12-31', freq='B')
#     tickers_to_simulate = [f'TICK{i:02d}' for i in range(50)]
#     all_ticker_dfs = []
#     for ticker in tickers_to_simulate:
#         drift = 0.0001 * (hash(ticker) % 20 - 10)
#         volatility = 0.005 + 0.03 * ((hash(ticker) % 10) / 10.0)
#         returns = drift + volatility * np.random.randn(len(date_rng))
#         close_prices = 100 * (1 + returns).cumprod()
#         temp_df = pd.DataFrame({'Adj Close': close_prices}, index=date_rng)
#         temp_df['Ticker'] = ticker
#         all_ticker_dfs.append(temp_df)
#     long_format_df = pd.concat(all_ticker_dfs).reset_index().rename(columns={'index': 'Date'}).set_index(['Ticker', 'Date'])

#     # Now you can call the inspector function directly
#     _ = inspect_ticker_data(long_format_df, ticker='TICK05', period='6M')

In [24]:
inspect_ticker_data(df_OHLCV, ticker='JAAA', period='3M')

### Sanity Check for Ticker: `JAAA` | Period: `3M`

**1. Price Performance Calculation:**

   - Start Date (2025-06-23): 50.06
   - End Date   (2025-09-22): 50.75
   - Final Normalized Price: (50.75 / 50.06) = 1.0137



**2. Sharpe Ratio Calculation:**

   - Number of Trading Days: 64
   - Mean of Daily Returns: 0.000217
   - Std Dev of Daily Returns: 0.000486
   - Daily Sharpe Ratio: (0.000217 / 0.000486) = 0.4457
   - Annualized Sharpe Ratio: (0.4457 * sqrt(252)) = 7.0759



**3. Underlying Data Sample:**

,Adj Close,Daily Return
Date,,
2025-06-23,50.0627,NaN
2025-06-24,50.0430,-0.000394
2025-06-25,50.0035,-0.000789
2025-06-26,50.0627,0.001184
2025-06-27,50.0923,0.000591


   ...


,Adj Close,Daily Return
Date,,
2025-09-16,50.67,-0.000395
2025-09-17,50.68,0.000197
2025-09-18,50.71,0.000592
2025-09-19,50.73,0.000394
2025-09-22,50.75,0.000394


,Adj Close,Daily Return
Date,,
2025-06-23,50.0627,NaN
2025-06-24,50.0430,-0.000394
2025-06-25,50.0035,-0.000789
2025-06-26,50.0627,0.001184
2025-06-27,50.0923,0.000591
2025-06-30,50.0923,0.000000
2025-07-01,50.1350,0.000852
2025-07-02,50.0953,-0.000792
2025-07-03,50.1251,0.000595


In [ ]:
inspect_ticker_data(df_OHLCV, ticker='IREN', period='3M')

In [ ]:
inspect_ticker_data(df_OHLCV, ticker='GOOG', period='3M')

In [18]:
_ticker = 'IREN'
_ticker_df = df_OHLCV.loc[_ticker].copy()

In [20]:
from pathlib import Path

download_path = Path.home() / "Downloads" / "IREN.csv"
_ticker_df.to_csv(download_path, index=True)

In [40]:
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import pprint 

def plot_interactive_performers_widget(df_ohlcv, 
                                       default_metric='Price',
                                       default_period='1Y', 
                                       default_rank_start=1,
                                       default_rank_end=10):
    """
    Creates a robust interactive plot using ipywidgets for state management.
    Metrics include Price, Sharpe, and Return/ATR.
    (Final corrected version).
    """
    # --- 1. Data Preparation: Reshape & Calculate Intermediate Values ---
    print("Reshaping data and calculating intermediate metrics (ATR)...")
    
    if not isinstance(df_ohlcv.index, pd.MultiIndex):
        raise ValueError("The input DataFrame does not have the expected 'long' format with a (Ticker, Date) MultiIndex.")
    
    df_close = df_ohlcv['Adj Close'].unstack(level=0)
    df_high = df_ohlcv['Adj High'].unstack(level=0)
    df_low = df_ohlcv['Adj Low'].unstack(level=0)
    df_close.index = pd.to_datetime(df_close.index)
    df_high.index = pd.to_datetime(df_high.index)
    df_low.index = pd.to_datetime(df_low.index)

    high_low = df_high - df_low
    high_prev_close = abs(df_high - df_close.shift(1))
    low_prev_close = abs(df_low - df_close.shift(1))
    tr = np.maximum(high_low, np.maximum(high_prev_close, low_prev_close))
    df_atr = tr.ewm(alpha=1/14, adjust=False).mean()
    
    print("Data successfully reshaped and ATR calculated.")
    
    # --- 2. Pre-computation Phase ---
    print("Starting pre-computation for all metrics across all periods...")
    end_date = df_close.index.max()
    metrics = ['Price', 'Sharpe', 'Return/ATR']
    periods = {
        '1D': end_date - pd.DateOffset(days=1), '5D': end_date - pd.DateOffset(days=5),
        '3M': end_date - pd.DateOffset(months=3), '6M': end_date - pd.DateOffset(months=6),
        'YTD': datetime(end_date.year, 1, 1), '1Y': end_date - pd.DateOffset(years=1),
        '5Y': end_date - pd.DateOffset(years=5),
    }
    all_rankings = {metric: {} for metric in metrics}
    period_normalized_data = {}
    for label, start_date in periods.items():
        period_close = df_close.loc[start_date:end_date]
        period_atr = df_atr.loc[start_date:end_date]
        if period_close.empty or len(period_close) < 2: continue
        first_prices, last_prices = period_close.bfill().iloc[0], period_close.ffill().iloc[-1]
        all_rankings['Price'][label] = (last_prices / first_prices).dropna().sort_values(ascending=False).index.tolist()
        daily_returns = period_close.pct_change()
        mean_returns, std_returns = daily_returns.mean(), daily_returns.std()
        sharpe_ratio = (mean_returns / std_returns).fillna(0)
        all_rankings['Sharpe'][label] = (sharpe_ratio * np.sqrt(252)).sort_values(ascending=False).index.tolist()
        mean_atr = period_atr.mean()
        return_on_atr = (mean_returns / mean_atr).fillna(0)
        all_rankings['Return/ATR'][label] = return_on_atr.sort_values(ascending=False).index.tolist()
        period_normalized_data[label] = period_close.div(first_prices)
    print("Pre-computation finished.")

    # --- 3. Create ipywidgets Controls & Data Holder ---
    rank_options = [1, 5, 10, 20, 30, 40, 50, 75, 100]
    max_traces = 50 
    metric_dropdown = widgets.Dropdown(options=metrics, value=default_metric, description='Metric:')
    period_dropdown = widgets.Dropdown(options=list(periods.keys()), value=default_period, description='Period:')
    rank_start_dropdown = widgets.Dropdown(options=rank_options, value=default_rank_start, description='Rank Start:')
    rank_end_dropdown = widgets.Dropdown(options=rank_options, value=default_rank_end, description='Rank End:')
    update_button = widgets.Button(description="Update Chart", button_style='primary')
    ticker_list_output = widgets.Output()
    _currently_displayed_tickers = []

    # --- 4. Create a FigureWidget ---
    fig = go.FigureWidget()
    for i in range(max_traces):
        fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name=f'placeholder_{i}', visible=False, showlegend=False))

    # --- 5. Define the Update Logic ---
    def update_plot(button_click):
        metric, period, rank_start, rank_end = metric_dropdown.value, period_dropdown.value, rank_start_dropdown.value, rank_end_dropdown.value
        ticker_list_output.clear_output()
        if rank_start > rank_end:
            with ticker_list_output: print("Error: 'Rank Start' must be less than or equal to 'Rank End'.")
            return
        ranked_tickers = all_rankings[metric].get(period, [])
        tickers_to_display = ranked_tickers[rank_start-1:rank_end]
        norm_data = period_normalized_data.get(period)

        with fig.batch_update():
            for i in range(max_traces):
                trace = fig.data[i]
                
                # --- THIS IS THE CORRECTED LOGIC ---
                # First, check if there's a ticker available for this trace index
                if i < len(tickers_to_display) and norm_data is not None:
                    ticker = tickers_to_display[i]
                    # Second, check if that ticker actually has data in the norm_data frame
                    if ticker in norm_data.columns:
                        trace.x, trace.y, trace.name = norm_data[ticker].index, norm_data[ticker], ticker
                        trace.visible, trace.showlegend = True, True
                    else: # Handle cases where a ticker was ranked but has no data
                        trace.visible, trace.showlegend = False, False
                else: # If no ticker for this trace index, hide it
                    trace.visible, trace.showlegend = False, False

        _currently_displayed_tickers.clear()
        _currently_displayed_tickers.extend(tickers_to_display)
        with ticker_list_output:
            print("Currently Displayed Tickers:")
            pprint.pprint(tickers_to_display, width=120, compact=True) 
    
    # --- 6. Set up Initial View and Layout ---
    update_plot(None)
    fig.update_layout(title_text='Top Performing Tickers (Normalized to 1)', xaxis_title='Date', yaxis_title='Normalized Price (Start = 1)', hovermode='x unified', legend_title_text='Tickers', height=700, margin=dict(t=50))
    fig.add_hline(y=1, line_width=1, line_dash="dash", line_color="grey")
    update_button.on_click(update_plot)
    
    # --- 7. Display the UI ---
    controls = widgets.HBox([metric_dropdown, period_dropdown, rank_start_dropdown, rank_end_dropdown, update_button])
    ui_container = widgets.VBox([controls, ticker_list_output], layout=widgets.Layout(margin='10px 0 20px 0'))
    display(ui_container, fig)
    return _currently_displayed_tickers

In [ ]:
# --- NEW: Initialize the variable before the call ---
plotted_tickers = [] 

# Call the final function and capture the returned list
# The 'plotted_tickers' list will be updated in place every time you click the button.
plotted_tickers = plot_interactive_performers_widget(
    df_OHLCV, 
    default_metric='Sharpe', 
    default_period='1Y', 
    default_rank_start=20, 
    default_rank_end=40
)

# --- NEW: Add a comment explaining how to use the variable ---
# After interacting with the plot and clicking "Update Chart", you can run a separate cell
# to see the contents of 'plotted_tickers'.
# For example, in a new cell, just type:
# print(plotted_tickers)

print("\n--- After the plot, the 'plotted_tickers' variable holds the last displayed list: ---")
print(plotted_tickers)

Reshaping data and calculating intermediate metrics (ATR)...
Data successfully reshaped and ATR calculated.
Starting pre-computation for all metrics across all periods...
Pre-computation finished.


FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'ESLT',
              'showlegend': True,
              'type': 'scatter',
              'uid': '2e306d78-65ea-4ac5-af98-b486506b29ff',
              'visible': True,
              'x': array([datetime.datetime(2024, 9, 23, 0, 0),
                          datetime.datetime(2024, 9, 24, 0, 0),
                          datetime.datetime(2024, 9, 25, 0, 0), ...,
                          datetime.datetime(2025, 9, 18, 0, 0),
                          datetime.datetime(2025, 9, 19, 0, 0),
                          datetime.datetime(2025, 9, 22, 0, 0)], dtype=object),
              'y': array([1.        , 1.01230212, 1.01178231, ..., 2.57609409, 2.56212754,
                          2.61988396])},
             {'mode': 'lines',
              'name': 'SPSB',
              'showlegend': True,
              'type': 'scatter',
              'uid': '5d88aa47-cbb7-47bf-b87d-8b6644ef4868',
              'visible': True,
        


--- After the plot, the 'plotted_tickers' variable holds the last displayed list: ---
['ESLT', 'SPSB', 'SGHC', 'VRNA', 'SHLD', 'STIP', 'OKLO', 'VTIP', 'KTOS', 'IDCC', 'HOOD', 'LLYVK', 'CLS', 'JPIE', 'LLYVA', 'PLTR', 'RBLX', 'RKLB', 'SOFI', 'FLOT', 'KGC']


In [42]:
print(plotted_tickers)

['NIO', 'HL', 'EQX', 'UWMC', 'GRAB', 'QS', 'BBD', 'NMR', 'CDE', 'EBR', 'CX', 'SAN', 'LYG', 'WBD', 'AEG', 'SBSW', 'FPE', 'VALE', 'IREN', 'PDI', 'PGX', 'PSLV', 'IVZ', 'B', 'KGC', 'RIOT', 'ROIV', 'PSKY', 'ELAN', 'SOUN']
